# Transformation des variables text en num

**Ce code est destiné à transformer les réponses de l'utilisateur qui peuvent être à la fois numérique mais aussi textuelles en variables numériques.**

Voici les variables que nous récupérons des réponses de l'utilisateur : 
- Nom
- Prénom
- Adresse
- Distance du festival
- Date
- Catégorie de festival
- Sous-catégorie
- Taille du festival souhaité
- Budget
- Durée du festival

## 1. Adresse

**Le premier objectif est de transformer l'adresse postale en coordonnées géographqiues**

Le code est récupéré du site officiel de data.gouv pour réaliser la requête API : https://guides.data.gouv.fr/reutiliser-des-donnees/utiliser-les-api-geographiques/utiliser-lapi-adresse/geocoder-des-adresses-pratique.
Cela nous permet de transformer l'adresse postale au format : Numéro de rue Type de rue Nom de la rue Ville Pays Code postal (5 Avenue Le Chatelier Palaiseau France 91120)

In [ ]:
import requests
ADDOK_URL = 'http://api-adresse.data.gouv.fr/search/'
params = {
    'q': '24 Rue des Diables Bleus 73000 Chambéry',
    'limit': 5
}
response = requests.get(ADDOK_URL, params=params)
j = response.json()
if len(j.get('features')) > 0:
    first_result = j.get('features')[0]
    lon, lat = first_result.get('geometry').get('coordinates')
    first_result_all_infos = { **first_result.get('properties'), **{"lon": lon, "lat": lat}}
    print(first_result_all_infos)
else:
    print('No result')

{'label': '24 Rue des diables bleus 73000 Chambéry', 'score': 0.9657545454545454, 'housenumber': '24', 'id': '73065_1110_00024', 'name': '24 Rue des diables bleus', 'postcode': '73000', 'citycode': '73065', 'x': 927213.22, 'y': 6501342.72, 'city': 'Chambéry', 'context': '73, Savoie, Auvergne-Rhône-Alpes', 'type': 'housenumber', 'importance': 0.6233, 'street': 'Rue des diables bleus', 'lon': 5.91394, 'lat': 45.573853}


On améliore le code pour qu'il corresponde plus à nos demandes, à savoir retourner uniquement les coordonnées géographiques. On gère aussi les erreurs possibles et on utilise ce code comme une fonction.

In [18]:
import requests

def adresse_to_coordgeo(adresse):
    """Convertit une adresse en coordonnées géographiques en utilisant l'API de DataGouv."""
    ADDOK_URL = 'http://api-adresse.data.gouv.fr/search/'
    params = {
        'q': adresse,
        'limit': 1  # On ne veut que le premier résultat
    }
    
    try:
        response = requests.get(ADDOK_URL, params=params)
        response.raise_for_status()  # Vérifie que la requête a réussi
        j = response.json()
        
        # Vérifier si des résultats ont été trouvés
        if len(j.get('features')) > 0:
            first_result = j.get('features')[0]
            lon, lat = first_result.get('geometry').get('coordinates')
            first_result_all_infos = {**first_result.get('properties'), **{"lon": lon, "lat": lat}}
            return (lat, lon)
        else:
            print('No result')
            return None
    except requests.exceptions.RequestException as e:
        print(f"Erreur lors de l'appel à l'API : {e}")
        return None

In [ ]:
adresse_to_coordgeo('5 Avenue Le Chatelier Palaiseau France 91120')

True

On vérifie pour 10 adresses que les coordonnés géographiques sont justes : 

In [ ]:
Adresse = (("24 Rue des Diables Bleus, 73000 Chambéry, France",(45.57406997680664, 5.914342880249023)), 
("5 Avenue Victor Hugo, 75016 Paris, France", (48.86894226074219, 2.2836291790008545)), 
("10 Place de la Bourse, 33000 Bordeaux, France", (44.84135437011719, -0.5704167485237122)), 
("17 Rue Saint-Nicolas, 54000 Nancy, France", (48.68845748901367, 6.18630313873291)), 
("12 Rue du Château, 44000 Nantes, France", (47.21629333496094, -1.55164635181427)), 
("8 Rue de la Poulaillerie, 69002 Lyon, France", (45.76422882080078, 4.833735942840576)), 
("7 Rue de la Liberté, 21000 Dijon, France", (47.322166442871094, 5.037889003753662)), 
("15 Boulevard de la Croisette, 06407 Cannes, France", (43.5514204, 7.0216248)), 
("3 Rue St-Rome, 31000 Toulouse, France", (43.60270690917969, 1.44353449344635)), 
("11 Avenue Jean Médecin, 06000 Nice, France", (43.698753356933594, 7.269191265106201)))